## **Downloading/Processing Temperature Files from Landsat 8 & 9.**

## **Required Dependencies**

In [1]:

import json
import ee
ee.Initialize()
from pathlib import Path
import os
from glob import glob
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm
import seaborn as sns
import random


### **Path Directories**

In [2]:
root_path                           =  Path(os.getcwd())

landsat_8_data_path                 =  root_path / 'Landsat_8_Data/Unprocessed_Data/'
landsat_9_data_path                 =  root_path / 'Landsat_9_Data/Unprocessed_Data/'

landsat_8_processed_data_path       =  root_path / 'Landsat_8_Data/Processed_Data/'
landsat_9_processed_data_path       =  root_path / 'Landsat_9_Data/Processed_Data/'


landsat_8_unprocessed_tiff_files    =  glob(os.path.join(landsat_8_data_path, '*.tif'))
landsat_9_unprocessed_tiff_files    =  glob(os.path.join(landsat_9_data_path, '*.tif'))


print(os.path.exists(landsat_8_data_path), os.path.exists(landsat_9_data_path), 
os.path.exists(landsat_8_processed_data_path), os.path.exists(landsat_9_processed_data_path))

len(landsat_8_unprocessed_tiff_files),         len(landsat_9_unprocessed_tiff_files)

True True True True


(36, 14)

## **Utilities**
A set of helpful functions.

In [3]:
def landsat_8_DN2Temperature(landsat_8_unprocessed_tiff_files, landsat_8_processed_data_path):
    """

    A Simple Function to Convert the Digital Numbers to Temperature Values.
    Args:
    landsat_8_unprocessed_tiff_files   :  A glob.glob list of Source GeoTiff files.
    landsat_8_processed_data_path      :  The destination Path. 
    Return                             :  None - Simply read the files, extract the Band 10 and 11 Data, Write it to 
                                            a GeoTiff File and saved it at landsat_8_processed_data_path. 
    
    """
    for tif_file in tqdm(landsat_8_unprocessed_tiff_files):
    
        # lets get the filename
        file_name              =  (os.path.basename(tif_file)).split(os.path.extsep)[0]
    
    
        # lets get the image id to get the constants info
        folder_img_id          =  '_'.join(file_name.split('_')[2:])
    
        img_id                 =  f'LANDSAT/LC08/C01/T1_TOA/{folder_img_id}'
    
        # Next, Easy - Fetch the Constants from Earth Engine
        image                  =  ee.Image(img_id)
        K1_constant_band_10    =  image.get('K1_CONSTANT_BAND_10').getInfo()
        K2_constant_band_10    =  image.get('K2_CONSTANT_BAND_10').getInfo()
        Radiance_Mult_Bn_10    =  image.get('RADIANCE_MULT_BAND_10').getInfo()
        Radiance_add_Bnd_10    =  image.get('RADIANCE_ADD_BAND_10').getInfo()
    
        K1_constant_band_11    =  image.get('K1_CONSTANT_BAND_11').getInfo()
        K2_constant_band_11    =  image.get('K2_CONSTANT_BAND_11').getInfo()
        Radiance_Mult_Bn_11    =  image.get('RADIANCE_MULT_BAND_11').getInfo()
        Radiance_add_Bnd_11    =  image.get('RADIANCE_ADD_BAND_11').getInfo()
    
        # Next, lets read the image using Rasterio and extract the band 10, band 11 data. 
        with rasterio.open(tif_file) as src_fil:
            Bn_10_data                  =  src_fil.read(10)
            Bn_11_data                  =  src_fil.read(11)
    
            # lets define the temperature conversion for band 10/11
            temperature_conversion_B10  =  K2_constant_band_10 / (np.log(K1_constant_band_10 / (Bn_10_data * Radiance_Mult_Bn_10 + Radiance_add_Bnd_10) + 1))
            temperature_conversion_B11  =  K2_constant_band_11 / (np.log(K1_constant_band_11 / (Bn_11_data * Radiance_Mult_Bn_11 + Radiance_add_Bnd_11) + 1))
    
            # lets save the temperature data as a new tiff file
            profile                     =   src_fil.profile
            profile.update(dtype=rasterio.float32, count=2)
    
            dest_filename               =  '_'.join(file_name.split('_')[:2]) + '_Temperature_' + '_'.join(file_name.split('_')[2:]) + '.tif'
            dest_file_path              =  os.path.join(landsat_8_processed_data_path, dest_filename)

            
            # Lets write the transformed temperature data to GeoTiff files. 
            with rasterio.open(dest_file_path, 'w', **profile) as dest_file:
                
                dest_file.write(temperature_conversion_B10.astype(rasterio.float32), 1)
                dest_file.write(temperature_conversion_B11.astype(rasterio.float32), 2)


def landsat_9_DN2Temperature(landsat_9_unprocessed_tiff_files, landsat_9_processed_data_path):
    """

    A Simple Function to Convert the Digital Numbers to Temperature Values.
    Args:
    landsat_9_unprocessed_tiff_files  :  A glob.glob list of Source GeoTiff files.
    landsat_9_processed_data_path     :  The destination Path. 
    Return                            :  None - Simply read the files, extract the Band 10 and 11 Data, Write it to 
                                            a GeoTiff File and saved it at landsat_8_processed_data_path. 
    
    """
    for L9_tiff_files in tqdm(landsat_9_unprocessed_tiff_files):

        # lets get the filename
        file_name              =  (os.path.basename(L9_tiff_files)).split(os.path.extsep)[0]

        
        # lets get the image id to get the constants info
        folder_img_id          =  '_'.join(file_name.split('_')[2:])
        img_id                 =  f'LANDSAT/LC09/C02/T1_TOA/{folder_img_id}'
        
    
        # Next, Easy - Fetch the Constants from Earth Engine
        image                  =  ee.Image(img_id)
        K1_constant_band_10    =  image.get('K1_CONSTANT_BAND_10').getInfo()
        K2_constant_band_10    =  image.get('K2_CONSTANT_BAND_10').getInfo()
        Radiance_Mult_Bn_10    =  image.get('RADIANCE_MULT_BAND_10').getInfo()
        Radiance_add_Bnd_10    =  image.get('RADIANCE_ADD_BAND_10').getInfo()
    
        K1_constant_band_11    =  image.get('K1_CONSTANT_BAND_11').getInfo()
        K2_constant_band_11    =  image.get('K2_CONSTANT_BAND_11').getInfo()
        Radiance_Mult_Bn_11    =  image.get('RADIANCE_MULT_BAND_11').getInfo()
        Radiance_add_Bnd_11    =  image.get('RADIANCE_ADD_BAND_11').getInfo()

        
        # Next, lets read the image using Rasterio and extract the band 10, band 11 data. 
        with rasterio.open(L9_tiff_files) as src_file:
            Bn_10_data                  =  src_file.read(10)
            Bn_11_data                  =  src_file.read(11)
          
    
            # lets define the temperature conversion for band 10/11
            temperature_conversion_B10  =  K2_constant_band_10 / (np.log(K1_constant_band_10 / (Bn_10_data * Radiance_Mult_Bn_10 + Radiance_add_Bnd_10) + 1))
            temperature_conversion_B11  =  K2_constant_band_11 / (np.log(K1_constant_band_11 / (Bn_11_data * Radiance_Mult_Bn_11 + Radiance_add_Bnd_11) + 1))
    
            # lets save the temperature data as a new tiff file
            profile                     =   src_file.profile
            profile.update(dtype=rasterio.float32, count=2)
    
            dest_filename               =  '_'.join(file_name.split('_')[:2]) + '_Temperature_' + '_'.join(file_name.split('_')[2:]) + '.tif'
            dest_file_path              =  os.path.join(landsat_9_processed_data_path, dest_filename)
        
    
            
            # Lets write the transformed temperature data to GeoTiff files. 
            with rasterio.open(dest_file_path, 'w', **profile) as dest_file:
                
                dest_file.write(temperature_conversion_B10.astype(rasterio.float32), 1)
                dest_file.write(temperature_conversion_B11.astype(rasterio.float32), 2)
           



### **DN To Temperature Conversion**

#### **Landsat 8**

In [4]:
landsat_8_DN2Temperature(landsat_8_unprocessed_tiff_files, landsat_8_processed_data_path)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [02:39<00:00,  4.43s/it]


#### **Landsat 9**

In [ ]:
landsat_9_DN2Temperature(landsat_9_unprocessed_tiff_files, landsat_9_processed_data_path)

 14%|██████████████████▌                                                                                                               | 2/14 [00:11<01:09,  5.78s/it]

### **Processed Data Path**

In [ ]:
landsat_8_processed_tiff_files         =   glob(os.path.join(landsat_8_processed_data_path, '*.tif'))
landsat_9_processed_tiff_files         =   glob(os.path.join(landsat_9_processed_data_path, '*.tif'))
len(landsat_8_processed_tiff_files),       len(landsat_9_processed_tiff_files)

## **Data Visualization**

### **Landsat 8 Visualiztion**

In [ ]:
for L8_tif_file in landsat_8_processed_tiff_files:

    complete_l8_tiff_files = [
                            'Lake_Champlain_Temperature_LC08_014029_20210810.tif', 'Lake_Champlain_Temperature_LC08_014029_20210826.tif',
                            'Lake_Champlain_Temperature_LC08_014029_20210911.tif', 'Lake_Champlain_Temperature_LC08_014029_20211013.tif', 
                            'Lake_Champlain_Temperature_LC08_014029_20211029.tif',
                         ]
    
    if os.path.basename(L8_tif_file) not in complete_l8_tiff_files: continue
    
    with rasterio.open(L8_tif_file, 'r') as src_file:
        B10_temp  = src_file.read(1)
        B11_temp  = src_file.read(2)
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    img1 = axs[0].imshow(B10_temp, cmap='hot',  interpolation='none')
    img2 = axs[1].imshow(B11_temp, cmap='hot',  interpolation='none')
    plt.colorbar(img1, ax=axs[0])
    plt.colorbar(img2, ax=axs[1])
    plot_title = os.path.basename(L8_tif_file).replace('Lake_Champlain_Temperature_', '').replace('.tif', '')
    axs[0].set_title(f'B10 Temperature Data of File {plot_title}', fontsize = 10, fontweight='bold')
    axs[1].set_title(f"B11 Temperature Data of File {plot_title}", fontsize = 10, fontweight='bold')
    plt.show()

### **Landsat 9 Visualization**

In [ ]:
for L9_tif_file in landsat_9_processed_tiff_files:

    complete_L9_tiff_files = [
                            'Lake_Champlain_Temperature_LC09_013029_20211104.tif', 'Lake_Champlain_Temperature_LC09_014029_20211109.tif',
                            'Lake_Champlain_Temperature_LC09_014029_20211114.tif', 'Lake_Champlain_Temperature_LC09_014029_20211224.tif', 
                         ]
    
    if os.path.basename(L9_tif_file) not in complete_L9_tiff_files: continue
    
    with rasterio.open(L9_tif_file, 'r') as src_file:
        B10_temp  = src_file.read(1)
        B11_temp  = src_file.read(2)
    fig, axs = plt.subplots(1, 2, figsize=(12, 6))
    img1 = axs[0].imshow(B10_temp, cmap='hot',  interpolation='none')
    img2 = axs[1].imshow(B11_temp, cmap='hot',  interpolation='none')
    plt.colorbar(img1, ax=axs[0])
    plt.colorbar(img2, ax=axs[1])
    plot_title = os.path.basename(L9_tif_file).replace('Lake_Champlain_Temperature_', '').replace('.tif', '')
    axs[0].set_title(f'B10 Temperature Data of File {plot_title}', fontsize = 10, fontweight='bold')
    axs[1].set_title(f"B11 Temperature Data of File {plot_title}", fontsize = 10, fontweight='bold')
    plt.show()